<a href="https://colab.research.google.com/github/jack-cao-623/python_learning/blob/main/Python_Decision_Tree%2C_Underfitting_and_Overfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Decision Tree, Underfitting and Overfitting

## Set up

In [3]:
# get kaggle data

# first, mount google drive

# copy .json file
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

# get data
! kaggle competitions download home-data-for-ml-course -f train.csv

100% 450k/450k [00:00<00:00, 1.53MB/s]
100% 450k/450k [00:00<00:00, 1.53MB/s]


In [4]:
# libraries needed
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [5]:
# get data into python
home_data = pd.read_csv('train.csv')
home_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
# descriptive stats about home_data
home_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [8]:
# specify features, X, and prediction target, y
y = home_data['SalePrice']
X = home_data[['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']]

In [22]:
# divide home_data into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

## Function to MAE for different tree depths

In [23]:
def get_mae(max_leaf_nodes, X_train, X_test, y_train, y_test):
  """
  returns mean absolute error (mae) given 
  maximum number of leaf nodes and training and test data
  """

  # specify model
  decision_tree_model = DecisionTreeRegressor(max_leaf_nodes = max_leaf_nodes, random_state = 1)
  
  # fit model
  decision_tree_model.fit(X_train, y_train)

  # get predicted values
  y_predicted = decision_tree_model.predict(X_test)

  # return MAE
  mae = mean_absolute_error(y_predicted, y_test)
  return mae


## For loop to iterate through various tree depths

In [24]:
get_mae(5, X_train, X_test, y_train, y_test)

35044.51299744237

In [31]:
# iterate through these tree depths
depths = [5, 50, 500, 5000]
mae_values = []

for depth in depths:
  mae_depth = get_mae(max_leaf_nodes = depth, X_train = X_train, X_test = X_test, y_train = y_train, y_test = y_test)
  mae_values.append(mae_depth)


In [32]:
# view different mae valus
print(mae_values)

[35044.51299744237, 27405.930473214907, 28380.917944156296, 29001.372602739724]


## Exercise from Kaggle

In [37]:
# set up so you match the exercises

# load data
home_data = pd.read_csv('train.csv')

# define features, X, and prediction target, y
X = home_data[['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']]
y = home_data[['SalePrice']]

# split X and y into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

# specify model
iowa_model = DecisionTreeRegressor(random_state = 1)

# fit model
iowa_model.fit(X_train, y_train)

# predicted y
y_predicted = iowa_model.predict(X_test)

# MAE
print(mean_absolute_error(y_predicted, y_test))

29652.931506849316


In [38]:
# function to get MAE given training and test data and tree depth
def get_mae(max_leaf_nodes, X_train, X_test, y_train, y_test):
  """
  get MAE given training and test data and tree depth
  """
  # specify model
  model = DecisionTreeRegressor(max_leaf_nodes = max_leaf_nodes, random_state = 0)

  # fit model
  model.fit(X_train, y_train)

  # make predictions
  y_predicted = model.predict(X_test)

  # get and return MAE
  mae = mean_absolute_error(y_predicted, y_test)
  return mae

In [39]:
# iterate through these tree depths
tree_depths = [5, 25, 50, 100, 250, 500]

# put MAE corresponding to above tree_depths in this list
mae_list = []

# iterate through each tree depth to get corresponding MAE
for depth in tree_depths:
  mae = get_mae(max_leaf_nodes = depth, X_train = X_train, X_test = X_test, y_train = y_train, y_test = y_test)
  mae_list.append(mae)

In [40]:
print(mae_list)

[35044.51299744237, 29016.41319191076, 27405.930473214907, 27282.50803885739, 27893.822225701646, 29454.18598068598]


In [43]:
# view tree depth and corresponding MAE together
pd.DataFrame(tree_depths, mae_list)

,0
35044.512997,5
29016.413192,25
27405.930473,50
27282.508039,100
27893.822226,250
29454.185981,500


In [44]:
# todo above: make tree depth the index and pull index with smallest MAE

In [45]:
# specify final model
final_model = DecisionTreeRegressor(max_leaf_nodes = 100, random_state = 0)

# fit final model
final_model.fit(X, y)

DecisionTreeRegressor(max_leaf_nodes=100, random_state=0)